# MNIST Experiments
## Imports

In [2]:
from keras.datasets import mnist
import numpy as np

from lib.others import build_model
from lib.models import Ganomaly1d, Ganomaly2d

(images, labels), (_, _) = mnist.load_data()
indices = [index for index, label in enumerate(labels) if label == 8]

X = images[indices][:400]

X = X.reshape((-1, 1, 28, 28))

X = np.array(X, dtype=np.float32)

In [3]:
mnist_model = build_model(Ganomaly2d, 28, 50, 'mnist2')

In [4]:
mnist_model.fit(X)

  epoch    loss_dis    loss_gen    loss_gen_app    loss_gen_fra    loss_gen_lat    train_loss      dur
-------  ----------  ----------  --------------  --------------  --------------  ------------  -------
      1      0.3154   1125.6259         37.4846          0.7466          0.3408     1125.9413  11.7350
      2      0.1182   1123.1165         37.4089          0.7524          0.0965     1123.2348  13.1663
      3      0.0925   1122.5925         37.3923          0.7545          0.0684     1122.6849  12.2161
      4      0.0789   1122.2559         37.3815          0.7550          0.0545     1122.3348  13.1244
      5      0.0695   1121.9978         37.3730          0.7552          0.0519     1122.0673  10.9130
      6      0.0631   1121.7796         37.3659          0.7557          0.0481     1121.8427  11.4598
      7      0.0565   1121.5881         37.3596          0.7562          0.0436     1121.6446  10.6652
      8      0.0515   1121.4514         37.3551          0.7568          

<class 'lib.models.GanomalyNet'>[initialized](
  module_=Ganomaly2d(
    (l_fra): BCELoss()
    (l_app): L1Loss()
    (l_dis): L1Loss()
    (discriminator): DiscriminatorNet2d(
      (features): Sequential(
        (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): LeakyReLU(negative_slope=0.2, inplace=True)
        (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): LeakyReLU(negative_slope=0.2, inplace=True)
        (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (7): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (classifier): Sequential(
        (0): Conv2d(256, 600, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (Sigmoid): Sigmoid()
      )
    )